## Propensity Score Weighting in causaltune 


In [10]:
import os
import sys
import pandas as pd
import numpy as np
import warnings

from scipy.stats import betabinom

from sklearn.ensemble import RandomForestClassifier

root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "auto-causality"))

from causaltune import CausalTune
from causaltune.data_utils import CausalityDataset
from causaltune.datasets import generate_non_random_dataset

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



Causaltune effect estimation consists of multiple models that can / need to be fitted.

1. Propensity model to estimate treatment propensities from features $\mathbb{E}[T|X,W]$.
2. Outcome model to estimate outcomes from features $\mathbb{E}[Y|X,W]$
3. The final causal inference estimator which requires additional hyperparamter tuning.

In this notebook, we focu on the Propensity Score Weighting (1.).

There are four options to finding a propensity model.

1. **[Default:] use a dummy estimator.**
   - natural option for a computationally easy model / when perfect randomisation of the treatment is given 


2. **Letting AutoML fit the propoensity model,**
   - has all the advantages of using an elaborate propensity weighting model  


3. **supply a custom sklearn-compatible prediction model,**
   - for more flexibility in terms of propensity prediction model


4. **supply an array of custom propensities to treat.** 
   - can be used, e.g. with custom propensities to treat based on an optimisation procedure such as Thompson sampling when there is an expected benefit from treating some subjects with higher propensity than others

### Generate Data

In [36]:
cd = generate_non_random_dataset()
cd.preprocess_dataset()

In [38]:
# causaltune configuration
components_time_budget = 30
train_size = 0.7

target = cd.outcomes[0]

### 1. DEFAULT: Dummy propensity model


The dummy propensity model identifies a constant propensity to treat given by $\frac{\text{Treatment Group Size}}{\text{Total Sample Size}}  $.

In [39]:
ct = CausalTune(
    propensity_model='dummy',
    components_time_budget=components_time_budget,
    metric="energy_distance",
    train_size=train_size,
    verbose=0
)   
ct.fit(data=cd, outcome=target)

Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks
Initial configs: [{'estimator': {'estimator_name': 'backdoor.causaltune.models.NaiveDummy'}}, {'estimator': {'estimator_name': 'backdoor.causaltune.models.Dummy'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.SLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.dr.ForestDRLearner', 'min_propensity': 1e-06, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decrease': 0.0, 'max_samples': 0.45, 'min_balancedness_tol': 0.45, 'honest': True, 'subforest_size': 4}}, {'estimator': {'estimator_name': 'backdoor.econml.dml.CausalForestDML', 'drate': True, 'n_estimators': 100, 'criterion': 'mse', 'min_samples_split': 10, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decr

In [26]:
print(f'Propensity model: {ct.propensity_model}')

Propensity model: DummyClassifier()


Difference in means estimate (naive ATE):

In [29]:
print(f'{ct.scorer.naive_ate(cd.data[cd.treatment], cd.data[target])[0]:.5f}')

0.41014


Causaltune ATE estimate:

In [31]:
print(f'{ct.effect(ct.test_df).mean():.5f}')

0.38910


### 2. Propensity model estimation via AutoML

The propensity score weighting estimation via AutoML is as simple as selecting `propensity_model='auto'`. 

The computational intensity can then be adapted via supplying a `components_budget_time`.


In [32]:
ct = CausalTune(
    propensity_model='auto',
    components_time_budget=components_time_budget,
    metric="energy_distance",
    train_size=train_size,
    verbose=0
)   
ct.fit(data=cd, outcome=target)

Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks
Initial configs: [{'estimator': {'estimator_name': 'backdoor.causaltune.models.NaiveDummy'}}, {'estimator': {'estimator_name': 'backdoor.causaltune.models.Dummy'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.SLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.dr.ForestDRLearner', 'min_propensity': 1e-06, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decrease': 0.0, 'max_samples': 0.45, 'min_balancedness_tol': 0.45, 'honest': True, 'subforest_size': 4}}, {'estimator': {'estimator_name': 'backdoor.econml.dml.CausalForestDML', 'drate': True, 'n_estimators': 100, 'criterion': 'mse', 'min_samples_split': 10, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decr

In [33]:
print(f'Estimated ATE: {ct.effect(ct.test_df).mean():.5f}')

Estimated ATE: 0.51501


### 3. Propensity model estimation with a custom model

A custom propensity model that has an sklearn-style `fit` and `predict_proba` method can be supplied as a propensity model.

In [34]:
propensity_model = RandomForestClassifier()

ct = CausalTune(
    propensity_model=propensity_model,
    components_time_budget=components_time_budget,
    metric="energy_distance",
    train_size=train_size,
)   
ct.fit(data=cd, outcome=target)

Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks
Initial configs: [{'estimator': {'estimator_name': 'backdoor.causaltune.models.NaiveDummy'}}, {'estimator': {'estimator_name': 'backdoor.causaltune.models.Dummy'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.SLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.dr.ForestDRLearner', 'min_propensity': 1e-06, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decrease': 0.0, 'max_samples': 0.45, 'min_balancedness_tol': 0.45, 'honest': True, 'subforest_size': 4}}, {'estimator': {'estimator_name': 'backdoor.econml.dml.CausalForestDML', 'drate': True, 'n_estimators': 100, 'criterion': 'mse', 'min_samples_split': 10, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'min_impurity_decr

### 4. Supplying individual treatment propensities

In some settings such as uplift modelling, the experiment / study is based on heterogeneous treatment propensities known to the researcher / experimenter. An array of treatment propensities can be directly supplied to causaltune in the data instantiation of the `CausalityDataset`. This can, e.g. be done by 
```
cd = CausalityDataset(
    ...
    propensity_modifiers=[<individual_treatment_propensity_column_name>]
    ...
)
```
and then using the `passthrough_model` as follows

In [35]:
from causaltune.models.passthrough import passthrough_model

print(cd.data.head())
print(f'True propensities to treat: {cd.propensity_modifiers}')

propensity_model=passthrough_model(
    cd.propensity_modifiers, include_control=False
    )

ct = CausalTune(
    propensity_model=propensity_model,
    components_time_budget=components_time_budget,
    metric="energy_distance",
    train_size=train_size,
    verbose=0
)   
ct.fit(data=cd, outcome=target)

   T         Y  random        X1        X2        X3        X4        X5  \
0  1 -1.069278     0.0 -0.010210  0.694765  0.035349  0.353874  0.243489   
1  1 -0.076461     1.0 -0.383145 -0.012346  0.058882 -1.568806  0.213350   
2  0 -0.129938     0.0  1.164562  0.153863  0.608709 -0.608641  1.325249   
3  1  0.409181     1.0  2.742698  0.193850  1.758363 -1.909881 -0.560650   
4  0  0.746339     0.0  0.278876 -1.229170 -0.933592  0.229196  1.641256   

   propensity  
0    0.642568  
1    0.364888  
2    0.566726  
3    0.662177  
4    0.335791  
True propensities to treat: ['propensity']
Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks
Initial configs: [{'estimator': {'estimator_name': 'backdoor.causaltune.models.NaiveDummy'}}, {'estimator': {'estimator_name': 'backdoor.causaltune.models.Dummy'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.SLearner'}}, {'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'